# Project 1

[This dataset](https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/Proj1.csv) is adapted from the World Health Organization on Strokes (it's based on real data but is NOT REAL). Use this dataset to answer the following questions and perform the following tasks. Feel free to add extra cells as needed, but follow the structure listed here and clearly identify where each question is answered. Please remove any superflous code.

## Data Information

- `reg_to_vote`: 0 if no, 1 if yes.
- `age`: age of the patient in years.
- `hypertension`: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension.
- `heart_disease`: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease.
- `ever_married`: 0 if no, 1 if yes.
- `Residence_type`: 0 for Rural, 1 for Urban.
- `avg_glucose_level`: average glucose level in blood.
- `bmi`: body mass index.
- `smoking_status_smokes`, `smoking_status_formerly`: Whether or not the person smokes, or formerly smoked. If a person has 0's for both these columns, they never smoked.
- `stroke`: 1 if the patient had a stroke or 0 if not.
- `dog_owner`: 0 if no, 1 if yes.
- `er_visits`: number of recorded Emergency Room visits in lifetime.
- `raccoons_to_fight`: number of racoons the patient belives they could fight off at once.
- `fast_food_budget_month`: amount (in US dollars) spent on fast food per month.


## Part I: Logistic Regression
Build a logistic regression model to predict whether or not someone had a `stroke` based on **all** the other variables in the dataset.

1. Count the missing data per column, and remove rows with missing data (if any).
2. Use 10 fold cross validation for your model validation. Z-score your continuous variables only. Store both the train and test accuracies to check for overfitting. **Is the model overfit? How can you tell?**
3. After completing steps 1-2, fit another logistic regression model on ALL of the data (no model validation; but do z score) using the same predictors as before, and put the coefficients into a dataframe called `coef`.
4. print out a confusion matrix for the model you made in part 3. **What does this confusion matrix tell you about your model? How can you tell?**

## Part II: Data Exploration
The WHO has asked the following five questions, create **at least 1 ggplot graph per question** (using the above data + model when needed) to help answer each question, and **explicitly answer the question in a Markdown cell** below your graph. You may use other calculations to help support your answer but MUST pair it with a graph. Write your answer as if you were explaining it to a non-data scientist. You will be graded on the effectiveness and clarity of your graph, as well as the completeness, clarity, and correctness of your responses and justifications.

1. In this specific data set, do dog-owners over 50 have a higher average probability of stoke than non-dog owners who currently smoke? How can you tell?
2. What is the relationship between average blood glucose and BMI? Is the relationship between those two variables different for people who are and are not registered to vote? How can you tell?
3. Is your logistic regression model most accurate for people who make less than 30k, between 30-90k, or over 100k? Discuss the potential accuracy *and* ethical implications if your model *were* more accurate for different groups (you can use the full model from part I-3 to check accuracy; DO NOT create/fit new models for each income range.).
4. Which of the following variables is the strongest predictor of having a stroke (owning a dog, residence type, marriage, being registered to vote)? How were you able to tell?
5. Create a variable `er_visits_per_year` that calculates the # of visits to the ER that a person has had per year of life. Store this variable in your data frame (no need to include this variable in the previous logistic regression model). Is the # of ER visits per year different for stroke and non-stroke patients? How can you tell?

## PART I

In [73]:
# PART I
import numpy as np
import pandas as pd
from plotnine import *

from sklearn.linear_model import LogisticRegression # Logistic Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv
from sklearn.model_selection import cross_val_score # cross validation metrics
from sklearn.model_selection import cross_val_predict # cross validation metrics

In [25]:
data = pd.read_csv('https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/Proj1.csv')
data.head()

,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,smoking_status_smokes,smoking_status_formerly,reg_to_vote,dog_owner,raccoons_to_fight,fast_food_budget_month,income_in_k,er_visits
0,60.0,1.0,0.0,0.0,1.0,73.00,25.2,0,1,0,1.0,1.0,10.0,209.19,51.553645,9.0
1,4.0,0.0,0.0,0.0,0.0,110.15,17.1,0,0,0,0.0,1.0,13.0,176.46,45.405414,5.0
2,77.0,0.0,0.0,1.0,1.0,68.38,27.8,0,0,0,0.0,1.0,6.0,213.00,94.865174,8.0
3,37.0,0.0,0.0,1.0,1.0,95.08,30.1,0,0,0,1.0,1.0,12.0,161.90,84.123775,8.0
4,44.0,0.0,0.0,0.0,0.0,103.78,40.9,0,1,0,1.0,1.0,11.0,261.29,74.794596,11.0


In [26]:
data.isna().sum()

age                         13
hypertension                12
heart_disease               21
ever_married                 9
Residence_type              21
avg_glucose_level           31
bmi                        575
stroke                       0
smoking_status_smokes        0
smoking_status_formerly      0
reg_to_vote                 14
dog_owner                   21
raccoons_to_fight           27
fast_food_budget_month       8
income_in_k                 21
er_visits                   15
dtype: int64

In [86]:
data = data.dropna()
data = data.reset_index()

In [87]:
contpred=['age','avg_glucose_level','bmi','er_visits','raccoons_to_fight','fast_food_budget_month']
predictors=data.columns.drop('stroke')
# print(predictors)
X = data[contpred]
zscore = StandardScaler()
zX = zscore.fit_transform(X)
data.loc[:,contpred]=zX
data.head()

,level_0,index,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,smoking_status_smokes,smoking_status_formerly,reg_to_vote,dog_owner,raccoons_to_fight,fast_food_budget_month,income_in_k,er_visits
0,0,0,0.755732,1.0,0.0,0.0,1.0,-0.726799,-0.460635,0,1,0,1.0,1.0,0.012965,0.248335,51.553645,-0.021923
1,1,1,-1.706986,0.0,0.0,0.0,0.0,0.096425,-1.481171,0,0,0,0.0,1.0,0.756460,-0.548153,45.405414,-0.865736
2,2,2,1.503343,0.0,0.0,1.0,1.0,-0.829175,-0.133055,0,0,0,0.0,1.0,-0.978362,0.341052,94.865174,-0.232876
3,3,3,-0.255741,0.0,0.0,1.0,1.0,-0.237518,0.156726,0,0,0,1.0,1.0,0.508628,-0.902472,84.123775,-0.232876
4,4,4,0.052098,0.0,0.0,0.0,0.0,-0.044731,1.517441,0,1,0,1.0,1.0,0.260797,1.516195,74.794596,0.399983


In [88]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data[predictors],data['stroke'], test_size=0.2)

In [89]:
#10fold

kf=KFold(n_splits=10)

X=Xtrain
y=ytrain

logit = LogisticRegression(solver='lbfgs',max_iter=1000)

acc = []

for train, test in kf.split(X):
    X_train = X.iloc[train]
    X_test = X.iloc[test]
    y_train = y[train]
    y_test = y[test]
    model = logit.fit(X_train, y_train)
    acc.append(accuracy_score(y_test,model.predict(X_test)))
    
print(acc)

KeyError: '[1150, 1153, 1154, 1155, 1161, 1163, 1165, 1170, 1173, 1184, 1187, 1200, 1214, 1215, 1216, 1222, 1224, 1230, 1241, 1252, 1253, 1259, 1264, 1267, 1269, 1272, 1275, 1276, 1289, 1301, 1304, 1305, 1309, 1311, 1316, 1322, 1326, 1329, 1331, 1334, 1339, 1342, 1344, 1345, 1349, 1351, 1353, 1355, 1357, 1371, 1374, 1377, 1381, 1386, 1388, 1394, 1398, 1403, 1404, 1411, 1412, 1413, 1416, 1417, 1421, 1424, 1432, 1446, 1447, 1454, 1460, 1462, 1463, 1464, 1465, 1471, 1477, 1488, 1497, 1502, 1503, 1509, 1517, 1519, 1527, 1530, 1533, 1549, 1551, 1558, 1560, 1566, 1568, 1572, 1578, 1579, 1609, 1611, 1621, 1627, 1629, 1630, 1641, 1645, 1648, 1651, 1652, 1657, 1664, 1670, 1674, 1677, 1679, 1690, 1691, 1693, 1694, 1698, 1699, 1703, 1704, 1714, 1735, 1737, 1740, 1741, 1754, 1764, 1769, 1770, 1777, 1786, 1788, 1790, 1793, 1795, 1799, 1802, 1803, 1812, 1813, 1814, 1822, 1830, 1838, 1839, 1843, 1849, 1866, 1868, 1869, 1870, 1871, 1872, 1875, 1877, 1882, 1890, 1893, 1898, 1899, 1903, 1904, 1906, 1907

2. ANSWER HERE
https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter
https://stackoverflow.com/questions/53720684/use-kfold-split-to-fit-model-return-not-in-index
5. ANSWER HERE

## PART II

In [2]:
# PART II, 1

1. DISCUSSION + ANSWERE HERE

In [3]:
# PART II, 2

2. DISCUSSION + ANSWERE HERE

In [4]:
# PART II, 3

3. DISCUSSION + ANSWERE HERE

In [5]:
# PART II, 4

4. DISCUSSION + ANSWERE HERE

In [6]:
# PART II, 5

5. DISCUSSION + ANSWERE HERE